In [1]:
# Install the Ultralytics package
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 24.9 MB/s eta 0:00:00


In [2]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"sankezzz","key":"d1a0607265f521d28075fb028984b743"}'}

In [3]:
# Create a directory for the Kaggle API key and move the file there
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
# Set the correct file permissions
!chmod 600 ~/.kaggle/kaggle.json

# Download the dataset
!kaggle datasets download -d adilshamim8/people-detection

Dataset URL: https://www.kaggle.com/datasets/adilshamim8/people-detection
License(s): Attribution-NoDerivatives 4.0 International (CC BY-ND 4.0)
 98% 1.91G/1.94G [00:15<00:00, 83.5MB/s]
100% 1.94G/1.94G [00:15<00:00, 133MB/s] 


In [4]:
# Unzip the dataset
!unzip people-detection.zip -d /content/datasets/


Streaming output truncated to the last 5000 lines.
  inflating: /content/datasets/train/train/2010_005494_jpg.rf.1d921a649beb2274af3cdf6776aa358d.jpg  
  inflating: /content/datasets/train/train/2010_005494_jpg.rf.695eafd03e6ea75a5ad6c8b8ca830938.jpg  
  inflating: /content/datasets/train/train/2010_005494_jpg.rf.927c318646d7c0611b04697d69d0e3be.jpg  
  inflating: /content/datasets/train/train/2010_005512_jpg.rf.13f5d37fdf9cc51c16793383addced52.jpg  
  inflating: /content/datasets/train/train/2010_005512_jpg.rf.b6023800c60bed07e7c5704b21e6a463.jpg  
  inflating: /content/datasets/train/train/2010_005512_jpg.rf.fedb4d425ea255a3257648cf5d0a3384.jpg  
  inflating: /content/datasets/train/train/2010_005515_jpg.rf.7651507057a6f4de76e0ffaa0688036b.jpg  
  inflating: /content/datasets/train/train/2010_005515_jpg.rf.9b71380f9c44a20659c2c7fcf9a2f464.jpg  
  inflating: /content/datasets/train/train/2010_005515_jpg.rf.f2471546b5e7c031dd750b6bd0de3103.jpg  
  inflating: /content/datasets/train/tra

In [5]:
!find /content -maxdepth 4 -name "_annotations.csv" -print


/content/datasets/test/test/_annotations.csv
/content/datasets/valid/valid/_annotations.csv
/content/datasets/train/train/_annotations.csv


In [6]:
import os, re, glob, textwrap

# Base dirs jahan zip extract hota hai
candidates = ["/content/datasets", "/content/dataset", "/content"]
csv_paths = []
for base in candidates:
    if os.path.exists(base):
        csv_paths += glob.glob(os.path.join(base, "**", "_annotations.csv"), recursive=True)

if not csv_paths:
    raise SystemExit("❌ _annotations.csv not found. Check unzip path.")

# Heuristic: jis folder me _annotations.csv hai, wahi split folder hai
split_dirs = {}
for p in csv_paths:
    d = os.path.dirname(p)
    name = os.path.basename(d).lower()
    # try to infer split name from folder name
    if re.search(r"\btrain\b", name):
        split_dirs["train"] = d
    elif re.search(r"\bvalid|val\b", name):
        split_dirs["val"]   = d
    elif re.search(r"\btest\b", name):
        split_dirs["test"]  = d

# Fallback: parent folder name (handle nested train/train)
for p in csv_paths:
    if len(split_dirs)==3: break
    d = os.path.dirname(p)
    parent = os.path.basename(os.path.dirname(d)).lower()
    if "train" not in split_dirs and re.search(r"\btrain\b", parent):
        split_dirs["train"] = d
    if "val" not in split_dirs and re.search(r"\bvalid|val\b", parent):
        split_dirs["val"] = d
    if "test" not in split_dirs and re.search(r"\btest\b", parent):
        split_dirs["test"] = d

print("✅ Detected splits:")
for k,v in split_dirs.items():
    print(f"{k}: {v}")

assert "train" in split_dirs and "val" in split_dirs, "Need at least train & val splits."


✅ Detected splits:
test: /content/datasets/test/test
val: /content/datasets/valid/valid
train: /content/datasets/train/train


In [7]:
import pandas as pd

# train split ke annotations dekhte hain
csv_path = "/content/datasets/train/train/_annotations.csv"
df = pd.read_csv(csv_path)

print("👉 Columns in CSV:", df.columns.tolist())
print(df.head(5))


👉 Columns in CSV: ['filename', 'width', 'height', 'class', 'xmin', 'ymin', 'xmax', 'ymax']
                                            filename  width  height   class  \
0  2008_003132_jpg.rf.92f6223defec4f57f2d7b9cfa28...    500     375  person   
1  2008_003132_jpg.rf.92f6223defec4f57f2d7b9cfa28...    500     375  person   
2  2008_003132_jpg.rf.92f6223defec4f57f2d7b9cfa28...    500     375  person   
3  004574_jpg.rf.7c8cea69d7be45f58febcede26ef0c6e...    500     333  person   
4  004574_jpg.rf.7c8cea69d7be45f58febcede26ef0c6e...    500     333  person   

   xmin  ymin  xmax  ymax  
0   219    98   269   283  
1   114   124   155   263  
2    43   139    98   340  
3   145   118   229   333  
4   285   105   349   329  


In [8]:
import os
import pandas as pd
from tqdm import tqdm

root = "/content/datasets"
splits = ["train/train", "valid/valid", "test/test"]

for split in splits:
    img_dir = os.path.join(root, split)
    csv_file = os.path.join(img_dir, "_annotations.csv")
    labels_dir = os.path.join(img_dir, "labels")
    os.makedirs(labels_dir, exist_ok=True)

    df = pd.read_csv(csv_file)

    for img_name, group in tqdm(df.groupby("filename"), desc=f"Processing {split}"):
        txt_path = os.path.join(labels_dir, img_name.replace(".jpg", ".txt").replace(".png",".txt"))
        with open(txt_path, "w") as f:
            for _, row in group.iterrows():
                w, h = row["width"], row["height"]
                xmin, ymin, xmax, ymax = row["xmin"], row["ymin"], row["xmax"], row["ymax"]

                # YOLO normalized format
                x_center = ((xmin + xmax) / 2) / w
                y_center = ((ymin + ymax) / 2) / h
                bw = (xmax - xmin) / w
                bh = (ymax - ymin) / h

                # Only one class 'person' → id=0
                f.write(f"0 {x_center:.6f} {y_center:.6f} {bw:.6f} {bh:.6f}\n")

print("✅ Conversion done! labels created in each split/labels/")


Processing test/test: 100%|██████████| 738/738 [00:00<00:00, 2483.41it/s]

✅ Conversion done! labels created in each split/labels/


In [9]:
import glob

for split in splits:
    img_dir = os.path.join(root, split)
    n_imgs = len(glob.glob(os.path.join(img_dir, "*.jpg")))
    n_labels = len(glob.glob(os.path.join(img_dir, "labels", "*.txt")))
    print(f"{split}: images={n_imgs}, label-files={n_labels}")


train/train: images=15210, label-files=13278
valid/valid: images=1431, label-files=1370
test/test: images=760, label-files=738


In [10]:
yaml_text = """
train: /content/datasets/train/train
val: /content/datasets/valid/valid
test: /content/datasets/test/test

nc: 1  # number of classes
names: ['person']
"""

with open("/content/datasets/people.yaml", "w") as f:
    f.write(yaml_text)

print("✅ people.yaml created")
print(yaml_text)


✅ people.yaml created

train: /content/datasets/train/train
val: /content/datasets/valid/valid
test: /content/datasets/test/test

nc: 1  # number of classes
names: ['person']



In [11]:
!ls datasets/train/train/labels | head


000001_jpg.rf.896508b6d1874829ee5d32d736158d29.txt
000001_jpg.rf.c82cb40fa9d0494897fb9ce8157a3dc8.txt
000001_jpg.rf.fd11b57224ecddaec5af674c447c67f4.txt
000003_jpg.rf.27f5678c703b1852a2779889278e70d5.txt
000003_jpg.rf.7ec28422125b41630b7a6a5599793ac4.txt
000003_jpg.rf.a1715bd43b0534b15e1aea18d2e2fa19.txt
000007_jpg.rf.04684d55fa4b8a8d7583bb4377206486.txt
000007_jpg.rf.168e1d1bca0a8ed616f402612c39f11f.txt
000007_jpg.rf.a0ce77226b43d517d362df752ee6a089.txt
000010_jpg.rf.c1586482a24e8577f35044bbdade2483.txt


In [12]:
!ls /content/datasets/train/train | grep .csv
!ls /content/datasets/valid/valid | grep .csv
!ls /content/datasets/test/test | grep .csv


_annotations.csv
_annotations.csv
_annotations.csv


In [13]:
!ls datasets/train/train | grep "_annotations.csv"
!ls datasets/valid/valid | grep "_annotations.csv"
!ls datasets/test/test | grep "_annotations.csv"


_annotations.csv
_annotations.csv
_annotations.csv


In [14]:
import os
import pandas as pd

# Function to convert CSV to YOLO txt
def convert_csv_to_yolo(csv_path, images_dir, labels_dir, class_mapping):
    os.makedirs(labels_dir, exist_ok=True)
    df = pd.read_csv(csv_path)

    for filename, group in df.groupby("filename"):
        # make txt file with same name as image but .txt extension
        label_file = os.path.splitext(filename)[0] + ".txt"
        label_path = os.path.join(labels_dir, label_file)

        with open(label_path, "w") as f:
            for _, row in group.iterrows():
                class_id = class_mapping.get(row["class"], -1)
                if class_id == -1:  # skip unknown classes
                    continue

                img_w, img_h = row["width"], row["height"]
                xmin, ymin, xmax, ymax = row["xmin"], row["ymin"], row["xmax"], row["ymax"]

                # YOLO normalized format
                x_center = ((xmin + xmax) / 2) / img_w
                y_center = ((ymin + ymax) / 2) / img_h
                w = (xmax - xmin) / img_w
                h = (ymax - ymin) / img_h

                f.write(f"{class_id} {x_center} {y_center} {w} {h}\n")

    print(f"✅ Labels saved in {labels_dir}")

# Paths
train_csv = "datasets/train/train/_annotations.csv"
valid_csv = "datasets/valid/valid/_annotations.csv"
test_csv  = "datasets/test/test/_annotations.csv"

# Labels directories
train_labels = "datasets/train/train/labels"
valid_labels = "datasets/valid/valid/labels"
test_labels  = "datasets/test/test/labels"

# Only 'person' class = id 0
class_mapping = {"person": 0}

# Convert for each split
convert_csv_to_yolo(train_csv, "datasets/train/train", train_labels, class_mapping)
convert_csv_to_yolo(valid_csv, "datasets/valid/valid", valid_labels, class_mapping)
convert_csv_to_yolo(test_csv, "datasets/test/test", test_labels, class_mapping)


✅ Labels saved in datasets/train/train/labels
✅ Labels saved in datasets/valid/valid/labels
✅ Labels saved in datasets/test/test/labels


In [15]:
import os
import pandas as pd

# Paths
splits = ["train/train", "valid/valid", "test/test"]
class_map = {"person": 0}  # sirf ek class hai

for split in splits:
    csv_path = f"/content/datasets/{split}/_annotations.csv"
    df = pd.read_csv(csv_path)

    for _, row in df.iterrows():
        img_w, img_h = row["width"], row["height"]
        xmin, ymin, xmax, ymax = row["xmin"], row["ymin"], row["xmax"], row["ymax"]

        # Normalize
        x_center = ((xmin + xmax) / 2) / img_w
        y_center = ((ymin + ymax) / 2) / img_h
        w = (xmax - xmin) / img_w
        h = (ymax - ymin) / img_h

        class_id = class_map[row["class"]]

        # Save as .txt file with same name as image
        label_path = os.path.join(f"/content/datasets/{split}", os.path.splitext(row["filename"])[0] + ".txt")
        with open(label_path, "a") as f:  # append (multiple boxes per image)
            f.write(f"{class_id} {x_center} {y_center} {w} {h}\n")


In [16]:
!rm -rf runs/


In [17]:
!rm -rf /content/datasets/*.cache
!rm -rf /content/datasets/*/*.cache


In [18]:
from ultralytics import YOLO

# Pretrained YOLOv8 model load
model = YOLO("yolov8n.pt")  # nano (fastest)

# Train again
model.train(
    data="/content/datasets/people.yaml",
    epochs=40,
    imgsz=640,
    batch=48,
    device=0
)


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


Ultralytics 8.3.184 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=48, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=/content/datasets/people.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=50, erasing=0.4, exist_ok=False, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=640, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=train, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, perspective=0.0, plots=True, pose=12.0, pretrained=T

Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

Model summary: 129 layers, 3,011,043 parameters, 3,011,027 gradients, 8.2 GFLOPs

Transferred 319/355 items from pretrained weights
Freezing layer 'model.22.dfl.conv.weight'
AMP: running Automatic Mixed Precision (AMP) checks...


AMP: checks passed ✅
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 2682.3±916.4 MB/s, size: 185.8 KB)


...

                   all       1431      10660      0.813      0.651      0.741      0.468



50 epochs completed in 4.082 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 6.2MB
Optimizer stripped from runs/detect/train/weights/best.pt, 6.2MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.184 🚀 Python-3.12.11 torch-2.8.0+cu126 CUDA:0 (Tesla T4, 15095MiB)
Model summary (fused): 72 layers, 3,005,843 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 15/15 [00:18<00:00,  1.25s/it]


                   all       1431      10660      0.813      0.652      0.741      0.468
Speed: 0.2ms preprocess, 1.9ms inference, 0.0ms loss, 3.0ms postprocess per image
Results saved to runs/detect/train


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7aab24fad7f0>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.048048, 

In [19]:
!zip -r runs.zip runs


  adding: runs/ (stored 0%)
  adding: runs/detect/ (stored 0%)
  adding: runs/detect/train/ (stored 0%)
  adding: runs/detect/train/BoxPR_curve.png (deflated 18%)
  adding: runs/detect/train/confusion_matrix.png (deflated 34%)
  adding: runs/detect/train/results.png (deflated 8%)
  adding: runs/detect/train/val_batch2_pred.jpg (deflated 4%)
  adding: runs/detect/train/results.csv (deflated 60%)
  adding: runs/detect/train/labels.jpg (deflated 33%)
  adding: runs/detect/train/train_batch2.jpg (deflated 3%)
  adding: runs/detect/train/confusion_matrix_normalized.png (deflated 35%)
  adding: runs/detect/train/weights/ (stored 0%)
  adding: runs/detect/train/weights/last.pt (deflated 9%)
  adding: runs/detect/train/weights/best.pt (deflated 9%)
  adding: runs/detect/train/val_batch2_labels.jpg (deflated 5%)
  adding: runs/detect/train/train_batch12681.jpg (deflated 7%)
  adding: runs/detect/train/BoxF1_curve.png (deflated 17%)
  adding: runs/detect/train/train_batch1.jpg (deflated 3%)
  ad

In [20]:
from google.colab import files
files.download("runs.zip")


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
!zip -r datasets.zip datasets
from google.colab import files
files.download("datasets.zip")


Streaming output truncated to the last 5000 lines.
  adding: datasets/train/train/2009_003010_jpg.rf.53d08d1282e5a692dfb188a630b12d4f.txt (deflated 43%)
  adding: datasets/train/train/2008_003683_jpg.rf.022abe053e4ac49f43ffa65fb30f9af0.jpg (deflated 0%)
  adding: datasets/train/train/2009_002674_jpg.rf.817ef4a25f707ea1cca7902dd58b8587.jpg (deflated 1%)
  adding: datasets/train/train/2010_006050_jpg.rf.8d84dfca0fb7449bbffc2d372dd00b07.txt (deflated 13%)
  adding: datasets/train/train/000108_jpg.rf.cf69a1e17689a2a7998255f0ab592f31.txt (deflated 76%)
  adding: datasets/train/train/2008_003544_jpg.rf.67f0e419485259acbd4bfdf3103eb459.txt (deflated 50%)
  adding: datasets/train/train/0006180_jpg.rf.e3f52811b41d8887721259680a6c66cf.txt (deflated 46%)
  adding: datasets/train/train/2008_005396_jpg.rf.c6ad6e9b08c1f54bc0e95f3b815ccf4d.jpg (deflated 3%)
  adding: datasets/train/train/2007_005790_jpg.rf.53101b6c8eb44e993ca37a14f26cac9b.jpg (deflated 1%)
  adding: datasets/train/train/2008_008627_j

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>